In [1]:
# Test CUDA operation in PyTorch

import torch

x = torch.rand(3, 3).cuda()
print(x)

tensor([[0.7453, 0.1396, 0.5272],
        [0.7037, 0.1840, 0.2191],
        [0.7179, 0.1252, 0.5156]], device='cuda:0')


In [2]:
# Test CUDA availability
import torch

print(torch.cuda.is_available())

True


In [3]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4",
    torch_dtype="auto",
    device_map="auto",
    cache_dir="model_cache",
    offload_buffers=True,
)

########## flash attention is not WORKING
# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

CUDA extension not installed.
CUDA extension not installed.
c:\Users\samue\Desktop\Courses_Personal\1-Projects\Helium-Scraping\venv-heliume\Lib\site-packages\transformers\modeling_utils.py:4728: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
c:\Users\samue\Desktop\Courses_Personal\1-Projects\Helium-Scraping\venv-heliume\Lib\site-packages\accelerate\utils\modeling.py:1405: UserWarning: Current model requires 828138240 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [10]:
# default processer
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4",
    cache_dir="model_cache",
)

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                # "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
                "image": "screenshot.jpg",
            },
            {
                "type": "text",
                "text": "look closely and step by step extract all the product names with the respective price into a table. Donot repeat the same products again.",
            },
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=200)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [11]:
print(output_text[0])

Here is the table with the product names and their respective prices:

| Product Name | Price |
| --- | --- |
| Toffifee | 8.00 |
| Mulino Bianco Abbracci | 3.10 |
| Mulino Bianco Macine | 3.10 |
| Mars in Sonderpackung | 5.30 |
| Snickers in Sonderpackung | 5.30 |
| Twix in Sonderpackung | 5.85 |

Note: The prices are in Swiss Francs (CHF).
